In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math

In [2]:
# Util for showing omages
def show(img, size=75):
    plt.figure(figsize = (size,size))
    
    if len(img.shape) == 2: # if grayscale, set cmap
        plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    else: # if 3 channels, change from BGR to RGB
        plt.imshow(img[...,::-1])
        
    plt.show()

# Block Matching

In [12]:
from joblib import Parallel, delayed

def SAD(matA, matB):
  return np.sum(np.abs(matA - matB))

def SSD(matA, matB):
  return np.sum(np.square(matA - matB))

def get_row_matches(row, imgL, imgR, window, matching_fn):
  _, width, _ = imgL.shape
  # Window offset
  offset = int(window/2)
  result = []
  for colLeft in range(offset, width-offset):
    template = imgL[row-offset:row+offset +
                    1, colLeft-offset:colLeft+offset+1, :]

    bestMatchCol = -1
    bestMatchVal = math.inf

    for colRight in range(offset, width-offset):
      match = imgR[row-offset:row+offset+1,
                   colRight-offset:colRight+offset+1, :]
      matchVal = matching_fn(template, match)

      if matchVal < bestMatchVal:
        bestMatchCol = colRight
        bestMatchVal = matchVal
    
    # Append disparity
    if bestMatchCol != -1: 
      result.append(np.abs(colLeft - bestMatchCol))
    else: 
      result.append(0)
  return result


def get_block_matches(imgL, imgR, window, matching_fn):
  height, _, _ = imgL.shape
  offset = int(window/2)

  results = Parallel(n_jobs=10)(delayed(get_row_matches)(i, imgL, imgR, window,
                                              matching_fn) for i in range(offset, height-offset))
  return np.array(results)

(287, 380)
[[  0   0   0 ... 368 271 379]
 [  0   1   2 ...  11 114   0]
 [ 48  47  46 ...   0   0   0]
 ...
 [193 193 193 ...   1  24   0]
 [193 194 193 ...   1   0   0]
 [193 194 193 ...   2   0 143]]


In [19]:
imgL = cv.imread("./l1.png")
imgR = cv.imread("./r1.png")

for window in [3, 5, 9]:
  print("Window size: ", window)

  print("Calculating SAD ... ")
  disparity_map = get_block_matches(imgL, imgR, window, SAD)
  cv.imwrite(f"./block_matching/output_{window}_SAD.png", disparity_map)


  print("Calculating SSD ... ")
  disparity_map = get_block_matches(imgL, imgR, window, SSD)
  cv.imwrite(f"./block_matching/output_{window}_SSD.png", disparity_map)

  

Window size:  3
Calculating SAD: 
Calculating SSD: 
Window size:  5
Calculating SAD: 
Calculating SSD: 
Window size:  9
Calculating SAD: 
Calculating SSD: 
